## Youtube 댓글 크롤링


댓글 추출한 동영상 링크 : https://www.youtube.com/watch?v=l9_i-LmBK_Y

### 라이브러리

pandas와 googleapiclient.discovery의 build사용 <br><br>

이때, **googleapiclient.discovery의 build**는 Google API를 호출하기 위한 객체를 지원하는 라이브러리

In [1]:
import pandas
from googleapiclient.discovery import build

#### utc 시간대를 추후에 편집하기 용이하게 하고자 만듬

In [2]:
def utc_change(df) :
    Year = df[0:4]
    Month = df[5:7]
    Day = df[8:10]
    hour = df[11:13]
    minute = df[14:16]
    second = df[17:19]
    
    return Year+'-'+Month+'-'+Day +' '+hour+':'+minute+':'+second

**YouTube Data API**는 다양한 정보를 지원한다(자세한 정보는 https://developers.google.com/youtube/v3/docs/commentThreads/list)


현재 예시는 4가지 정보만 활용
***
**textDisplay :** 댓글의 내용<br><br>
**authorDisplayName:** 댓글 작성자<br><br>
**publishedAt:** 댓글 작성 시간<br><br>
**likeCount:** 좋아요 수<br><br>
***
## Youtube 크롤링 코드 


In [3]:
def give_me_comment(developerKey, videoid) :
    comments = list()
    api_obj = build('youtube', 'v3', developerKey= developerKey)
    response = api_obj.commentThreads().list(part='snippet,replies', videoId=videoid, maxResults=100).execute()

    while response:
        for item in response['items']:
            comment = item['snippet']['topLevelComment']['snippet']
            comments.append([comment['textDisplay'], comment['authorDisplayName'], comment['publishedAt'], comment['likeCount']])
 
            if item['snippet']['totalReplyCount'] > 0:
                for reply_item in item['replies']['comments']:
                    reply = reply_item['snippet']
                    comments.append([reply['textDisplay'], reply['authorDisplayName'], reply['publishedAt'], reply['likeCount']])
 
        if 'nextPageToken' in response:
            response = api_obj.commentThreads().list(part='snippet,replies', videoId=videoid, pageToken=response['nextPageToken'], maxResults=100).execute()
        else:
            break
    cols = ['comment', 'user name','Time','Thums Up']
    df = pandas.DataFrame(comments, columns = cols).sort_values('Thums Up', ascending = False)
    df['Time'] = df['Time'].apply(lambda x : utc_change(x))
    return df

#### 최종 결과 : 28개의 댓글(대댓글 포함)

In [4]:
df1 = give_me_comment('google api', 'l9_i-LmBK_Y')
df1

,comment,user name,Time,Thums Up
27,제네시스 전기차 디자인은 전체적으로 살짝 애매한? 매력이 있긴 한데 디자이너들이 머...,30ㆍ,2021-07-21 13:41:43,7
20,"<a href=""https://www.youtube.com/watch?v=l9_i-...",햄찌,2021-07-21 14:30:05,6
9,7천만원 중반대라니...... 전기차 언제쯤 살 수 있을지ㅠㅜ,교통사고에 관한 모든 이야기 [사고링크],2021-07-22 02:08:06,5
26,Gv70 제가 제일 좋아하는 제네시스 차종 중 하나죠.ㅎㅎ<br>이쁘게 나와네요.ㅎㅎ,fower hand,2021-07-21 13:41:56,4
0,"✔ 선물용 끝판왕 차량용 방향제! 마일론 방향제!\r<br><a href=""http...",남자들의 자동차 motorsjason,2021-07-21 13:39:14,3
22,가격이 eg80급일텐데... 메리트가 있으련지ㅜ 승차감 소재 고급감이야 아오5 ev...,R P,2021-07-21 14:16:00,3
24,형 목소리 존나섹시해,Park ST,2021-07-21 13:58:36,3
25,으야 eg80 보다 훨 낮네여 egv80 도 나왔으면 하네여 :),lolo holo,2021-07-21 13:58:31,3
16,"<a href=""https://www.youtube.com/watch?v=l9_i-...",GGOMA_꼬마 스튜디오,2021-07-21 21:51:29,3
7,깔끔하게 7500 만원 시작가 예상합니다.,도르마무,2021-07-22 03:52:16,2


In [7]:
# 엑셀 저장 코드
# df1.to_excel('sample.xlsx', header=['comment', 'user name','Time','Thums Up'], index=None)

#### 혹시 몰라 댓글 100개 이상인 링크 댓글 개수도 뽑아봤다.(https://www.youtube.com/watch?v=7vmdwdRJkIA)

In [5]:
give_me_comment('google api','7vmdwdRJkIA')

,comment,user name,Time,Thums Up
226,벤츠딜러가 용쓴다 ㅋㅋ,일론마스크,2021-07-19 22:28:37,47
151,평소에 영상 재밌게 보고는 있는데 이 영상은 뭔가 목적이 있어 보이는 영상같네요; ...,범쌤,2021-07-20 05:24:40,44
243,흠…정보의 왜곡이 좀 보이네요. 게다가 믿고 걸러야 하는 eqa 를..ㅊ,Ring-Ring Google,2021-07-19 20:38:11,42
174,도어손잡이 여는 부분에서.. 응??? 실제 주행 300Km 못간다는 부분에서 스킵....,jjshim1,2021-07-20 02:30:31,40
271,벤츠 전기차 가 차였여? EQC 1억짜리 그거 천대 팔고 짤리고 반토막짜리 E...,상칠당,2021-07-19 18:25:08,34
...,...,...,...,...
143,테슬라 팬보이들 부글부글,봉봉,2021-07-20 07:26:49,0
304,사람조은 임과장님 ㅎㅎ,피카츄,2021-07-19 16:15:51,0
305,마감이어케 중국산차보다못하냐....,minyoumeful,2021-07-19 16:10:51,0
140,과속보다는 단거리 단거리 끊어서 달려서 그럴것 같습니다.<br>단거리씩 자주 달리다...,자유민주주의,2021-07-20 15:12:40,0


#### 100개 이상도 잘나온다